In [55]:
import pandas as pd
import numpy as np
import csv
import sys
import os

my_dump = False  # if True, dump to pickle files in directory pickle/
my_pickle = True # if True, read from pickle files in directory pickle/

if my_dump == True and my_pickle == True:
    print("You can't dump while pickling.  You should dump before you pickle.")
    sys.exit()
    
if my_dump == True:
    try: 
        os.mkdir('pickle')
    except:
        print("pickle directory already exists")

# Load data.

In [56]:
if my_pickle == False:
    allergies = pd.read_csv("train/allergies.csv",
                           parse_dates=["START", "STOP"])
    care_plans = pd.read_csv("train/careplans.csv",
                            parse_dates=["START", "STOP"])
    conditions = pd.read_csv("train/conditions.csv",
                            parse_dates=["START", "STOP"])
    devices = pd.read_csv("train/devices.csv",
                         parse_dates=["START","STOP"])  
    encounters = pd.read_csv("train/encounters.csv",
                            parse_dates=["START", "STOP"])
    imaging_studies = pd.read_csv("train/imaging_studies.csv",
                                 parse_dates=["DATE"])
    immunizations = pd.read_csv("train/immunizations.csv",
                               parse_dates=["DATE"])
    medications = pd.read_csv("train/medications.csv",
                             parse_dates=["START","STOP"])
    observations = pd.read_csv("train/observations.csv",
                              parse_dates=["DATE"])
    #organizations = pd.read_csv("train/organizations.csv")

    #payers = pd.read_csv("train/payers.csv")
    #payer_transitions = pd.read_csv("train/payer_transitions.csv")
    procedures = pd.read_csv("train/procedures.csv",
                            parse_dates=["DATE"])
    #providers = pd.read_csv("train/providers.csv")
    #supplies = pd.read_csv("train/supplies.csv",
    #                      parse_dates=["DATE"])

In [57]:
if my_pickle == False:
    patients = pd.read_csv("train/patients.csv", 
                       parse_dates=["BIRTHDATE","DEATHDATE"])

In [58]:
if my_dump == True:
    import pickle
    pickle.dump(allergies, open("pickle/allergies.p","wb"))
    pickle.dump(care_plans, open("pickle/care_plans.p","wb"))
    pickle.dump(conditions, open("pickle/conditions.p","wb"))
    pickle.dump(devices, open("pickle/devices.p","wb"))
    pickle.dump(encounters, open("pickle/encounters.p","wb"))
    pickle.dump(imaging_studies, open("pickle/imaging_studies.p","wb"))
    pickle.dump(immunizations, open("pickle/immunizations.p","wb"))
    pickle.dump(medications, open("pickle/medications.p","wb"))
    pickle.dump(observations, open("pickle/observations.p","wb"))
    pickle.dump(procedures, open("pickle/procedures.p","wb"))
    pickle.dump(patients, open("pickle/patients.p","wb"))

In [59]:
if my_pickle == True:
    import pickle
    try: 
        allergies = pickle.load(open("pickle/allergies.p","rb"))
        care_plans = pickle.load(open("pickle/care_plans.p","rb"))
        conditions = pickle.load(open("pickle/conditions.p","rb"))
        devices = pickle.load(open("pickle/devices.p","rb"))
        encounters = pickle.load(open("pickle/encounters.p","rb"))
        imaging_studies = pickle.load(open("pickle/imaging_studies.p","rb"))
        immunizations = pickle.load(open("pickle/immunizations.p","rb"))
        medications = pickle.load(open("pickle/medications.p","rb"))
        observations = pickle.load(open("pickle/observations.p","rb"))
        procedures = pickle.load(open("pickle/procedures.p","rb"))
        patients = pickle.load(open("pickle/patients.p","rb"))
    except:
        print("You need to dump before you pickle.  Did you dump first?")


# IDs for Target

In [60]:
#diagnosed patients
covid_patient_ids = conditions[conditions.CODE == 840539006].PATIENT.unique()

# negative tests
negative_covid_patient_ids = frozenset(observations[(observations.CODE=='94531-1') & 
                                         (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique())
#hospitalized patients
inpatient_ids = frozenset(encounters[(encounters.REASONCODE == 840539006) & 
                           (encounters.CODE==1505002)].PATIENT)
# deceased patients
deceased_ids = frozenset(np.intersect1d(covid_patient_ids, patients[patients.DEATHDATE.notna()].Id))
# ventilated patients
vent_ids = frozenset(procedures[(procedures.CODE == 26763009) & 
                      (procedures.PATIENT.isin(covid_patient_ids))].PATIENT)
# ICU patients
icu_ids = frozenset(encounters[(encounters.CODE == 305351004) & 
                     (encounters.PATIENT.isin(covid_patient_ids))].PATIENT)

covid_patient_ids = frozenset(covid_patient_ids)

Calculate days hospitalized and days in ICU.

In [61]:
encounters["STOP_NEW"] = encounters["STOP"]
encounters["LENGTH"] = (encounters["STOP_NEW"] - encounters["START"]) / np.timedelta64(int(1), "D")

hospital_days = (
    encounters[["PATIENT","LENGTH","REASONCODE","CODE"]]
    .loc[(encounters.REASONCODE == 840539006) & 
        (encounters.CODE==1505002)]
    .groupby("PATIENT")["LENGTH"]
    .sum()
).to_dict()

icu_days = (
    encounters[["PATIENT","LENGTH","REASONCODE","CODE"]]
    .loc[(encounters.CODE == 305351004) & 
        (encounters.PATIENT.isin(covid_patient_ids))]
    .groupby("PATIENT")["LENGTH"]
    .sum()
).to_dict()


# Data Prep

## Patients

Calculate current age (for deceased, age if they were alive)

In [62]:
current_age = pd.to_datetime('2020-01-01') - patients["BIRTHDATE"]
patients["Age"] = current_age
patients["Age"] = patients["Age"] / np.timedelta64(int(1),'Y')

Remove people who died before January 1st, 2020.

In [63]:
dead_patients = patients.loc[patients["DEATHDATE"] < pd.to_datetime('2020-01-01'),
                            "Id"]


patients = patients.drop(
    patients.loc[patients["DEATHDATE"] < pd.to_datetime('2020-01-01'),
                "Id"]
    .index
)

patients_dict = patients.copy()
patients_dict = patients_dict.set_index("Id")
patients_dict = patients_dict.to_dict()

## Allergies

Drop allergies that start after 2020.

In [64]:
allergies = allergies.drop(
    allergies[allergies["START"] >= pd.to_datetime('2020-01-01')]
    .index
)

allergies = allergies.drop(
    allergies[allergies["PATIENT"].isin(dead_patients)]
    .index
)


Number of allergies for a patient.

In [65]:
num_allergies = allergies[["PATIENT", "START"]].groupby(["PATIENT"]).count()
vets_with_allergies = allergies["PATIENT"].unique()

num_allergies_dict = num_allergies.to_dict()

## Care Plans

Remove COVID-related care plans from care plans.  

In [66]:
care_plans = care_plans.drop(
    care_plans[care_plans["REASONCODE"]
               .isin(["840544004","840539006"])]
    .index
)

Remove care plans that start after 2020.

In [67]:
care_plans = care_plans.drop(
    care_plans[care_plans["START"] >= pd.to_datetime('2020-01-01')]
    .index
)

care_plans = care_plans.drop(
    care_plans[care_plans["PATIENT"].isin(dead_patients)]
    .index
)


Number of active care plans.  Time on active care plans.  Lifetime care plans.  Total length of time on care plans.

In [68]:
active_care_plans = (
  care_plans[["PATIENT","CODE","STOP"]]
    .loc[care_plans["STOP"].isnull()]
    .groupby("PATIENT")
    .count()
    .loc[:,"CODE"]
)

lifetime_care_plans = (
    care_plans[["PATIENT", "CODE", "STOP"]]
    .groupby("PATIENT")
    .count()
    .loc[:,"CODE"]
)

care_plans["STOP_NEW"] = care_plans["STOP"]
care_plans.loc[care_plans["STOP"].isnull(),"STOP_NEW"] = pd.to_datetime('2020-01-01')
care_plans["LENGTH"] = (care_plans["STOP_NEW"] - care_plans["START"]) / np.timedelta64(int(1), "Y")

active_care_plan_length = (
care_plans[["PATIENT","CODE","STOP","START","LENGTH"]]
    .loc[care_plans["STOP"].isnull()]
    .groupby("PATIENT")["LENGTH"]
    .max()
)

lifetime_care_plan_length = (
    care_plans[["PATIENT","LENGTH"]]
    .groupby("PATIENT")["LENGTH"]
    .sum()
)


## Conditions

Drop COVID-related conditions.

In [69]:
conditions = conditions.drop(
    conditions[conditions["DESCRIPTION"]
               .isin(["840544004","840539006"])]
    .index
)

Drop conditions beginning in 2020.

In [70]:
conditions = conditions.drop(
    conditions[conditions["START"] >= pd.to_datetime('2020-01-01')]
    .index
)

conditions = conditions.drop(
    conditions[conditions["PATIENT"].isin(dead_patients)]
    .index
)

Active and lifetime conditions.

In [71]:
active_conditions = (
  conditions[["PATIENT","CODE","STOP"]]
    .loc[conditions["STOP"].isnull()]
    .groupby("PATIENT")
    .count()
    .loc[:,"CODE"]
)

lifetime_conditions = (
    conditions[["PATIENT", "CODE", "STOP"]]
    .groupby("PATIENT")
    .count()
    .loc[:,"CODE"]
)

conditions["STOP_NEW"] = conditions["STOP"]
conditions.loc[conditions["STOP"].isnull(),"STOP_NEW"] = pd.to_datetime('2020-01-01')
conditions["LENGTH"] = (
    conditions["STOP_NEW"] - conditions["START"]) / np.timedelta64(int(1), "Y")

active_condition_length = (
conditions[["PATIENT","CODE","STOP","START","LENGTH"]]
    .loc[conditions["STOP"].isnull()]
    .groupby("PATIENT")["LENGTH"]
    .max()
)

lifetime_condition_length = (
    conditions[["PATIENT","LENGTH"]]
    .groupby("PATIENT")["LENGTH"]
    .sum()
)

## Devices

Drop devices before 2020.  Calculate time spent on a device in lifetime.

In [72]:
devices["STOP_NEW"] = devices["STOP"]
devices.loc[devices["STOP"].isnull(),"STOP_NEW"] = pd.to_datetime('2020-01-01')
devices["LENGTH"] = (
    devices["STOP_NEW"] - devices["START"]) / np.timedelta64(int(1), "Y")
devices = devices.drop(
    devices[devices["START"] >= pd.to_datetime('2020-01-01')]
    .index
)

device_lifetime_length = (
    devices[["PATIENT","LENGTH"]]
    .groupby("PATIENT")["LENGTH"]
    .sum()
)

devices = devices.drop(
    devices[devices["PATIENT"].isin(dead_patients)]
    .index
)

## Encounters

Drop encounters after 2020.  Calculate number of encounters, lifetime total cost of encounters, lifetime base cost, lifetime payer coverage.

In [73]:
if my_pickle == False:
    encounters = encounters.drop(
        encounters[encounters["START"] >= pd.to_datetime('2020-01-01T00:00Z')]
        .index
    )

    encounters = encounters.drop(
        encounters[encounters["PATIENT"].isin(dead_patients)]
        .index
    )

    encounters_count = (
        encounters[["PATIENT", "CODE"]]
        .groupby("PATIENT")
        .count()
        .loc[:,"CODE"]
    )

    encounters_lifetime_total_cost = (
        encounters[["PATIENT","TOTAL_CLAIM_COST"]]
        .groupby("PATIENT")["TOTAL_CLAIM_COST"]
        .sum()
    )

    encounters_lifetime_base_cost = (
        encounters[["PATIENT","BASE_ENCOUNTER_COST"]]
        .groupby("PATIENT")["BASE_ENCOUNTER_COST"]
        .sum()
    )

    encounters_lifetime_payer_coverage = (
        encounters[["PATIENT","PAYER_COVERAGE"]]
        .groupby("PATIENT")["PAYER_COVERAGE"]
        .sum()
    )

    def divide_sum_enc(df_sub):
        return df_sub["PAYER_COVERAGE"].sum()/float(df_sub["TOTAL_CLAIM_COST"].sum())

    encounters_lifetime_perc_covered = (
        encounters[["PATIENT", "PAYER_COVERAGE", "TOTAL_CLAIM_COST"]]
        .groupby("PATIENT").apply(divide_sum_enc)
    )



In [74]:
if my_dump == True:
    pickle.dump(encounters_count, open("pickle/encounters_count.p","wb"))
    pickle.dump(encounters_lifetime_total_cost, open("pickle/encounters_lifetime_total_cost.p","wb"))
    pickle.dump(encounters_lifetime_base_cost, open("pickle/encounters_lifetime_base_cost.p","wb"))
    pickle.dump(encounters_lifetime_payer_coverage, open("pickle/encounters_lifetime_payer_coverage.p","wb"))
    pickle.dump(encounters_lifetime_perc_covered, open("pickle/encounters_lifetime_perc_covered.p","wb"))

In [75]:
if my_pickle == True:
    encounters_count = pickle.load(open("pickle/encounters_count.p", "rb"))
    encounters_lifetime_total_cost = pickle.load(open("pickle/encounters_lifetime_total_cost.p", "rb"))
    encounters_lifetime_base_cost = pickle.load(open("pickle/encounters_lifetime_base_cost.p", "rb"))
    encounters_lifetime_payer_coverage = pickle.load(open("pickle/encounters_lifetime_payer_coverage.p", "rb"))
    encounters_lifetime_perc_covered = pickle.load(open("pickle/encounters_lifetime_perc_covered.p", "rb"))

## Imaging Studies

Drop imaging studies after 2020.  Lifetime number of imaging studies.

In [76]:
imaging_studies = imaging_studies.drop(
    imaging_studies[imaging_studies["DATE"] >= pd.to_datetime('2020-01-01')]
    .index
)

imaging_studies = imaging_studies.drop(
    imaging_studies[imaging_studies["PATIENT"].isin(dead_patients)]
    .index
)


imaging_studies_lifetime = (
    imaging_studies[["PATIENT", "ENCOUNTER"]]
    .groupby("PATIENT")
    .count()
    .loc[:,"ENCOUNTER"]
)

## Immunizations

Drop immunizations after 2020.  Lifetime number of immunizations and cost.

In [77]:
immunizations = immunizations.drop(
    immunizations[immunizations["DATE"] >= pd.to_datetime('2020-01-01')]
    .index
)

immunizations = immunizations.drop(
    immunizations[immunizations["PATIENT"].isin(dead_patients)]
    .index
)

immunizations_lifetime = (
    immunizations[["PATIENT", "CODE"]]
    .groupby("PATIENT")
    .count()
    .loc[:,"CODE"]
)

immunizations_lifetime_cost = (
    immunizations[["PATIENT", "BASE_COST"]]
    .groupby("PATIENT")["BASE_COST"]
    .sum()
)

## Medications

Remove medications after 2020.  Calculate lifetime medications, cost, and length.

In [78]:
if my_pickle == False:
    medications["STOP_NEW"] = medications["STOP"]
    medications.loc[medications["STOP"].isnull(),"STOP_NEW"] = pd.to_datetime('2020-01-01')
    medications["LENGTH"] = (
        medications["STOP_NEW"] - medications["START"]) / np.timedelta64(int(1), "W")
    medications = medications.drop(
        medications[medications["START"] >= pd.to_datetime('2020-01-01')]
        .index
    )

    medications = medications.drop(
        medications[medications["PATIENT"].isin(dead_patients)]
        .index
    )

    medications_lifetime = (
        medications[["PATIENT", "CODE"]]
        .groupby("PATIENT")
        .count()
        .loc[:,"CODE"]
    )

    medications_lifetime_cost = (
        medications[["PATIENT", "TOTALCOST"]]
        .groupby("PATIENT")["TOTALCOST"]
        .sum()
    )

    def divide_sum_med(df_sub):
        return df_sub["PAYER_COVERAGE"].sum()/float(df_sub["BASE_COST"].sum())

    medications_lifetime_perc_covered = (
        medications[["PATIENT", "PAYER_COVERAGE", "BASE_COST"]]
        .groupby("PATIENT").apply(divide_sum_med)
    )


    medications_lifetime_length = (
        medications[["PATIENT", "LENGTH"]]
        .groupby("PATIENT")["LENGTH"]
        .sum()
    )

    medications_lifetime_dispenses = (
        medications[["PATIENT", "DISPENSES"]]
        .groupby("PATIENT")["DISPENSES"]
        .sum()
    )
    medications_active = (
      medications[["PATIENT","CODE","STOP"]]
        .loc[medications["STOP"].isnull()]
        .groupby("PATIENT")
        .count()
        .loc[:,"CODE"]
    )



In [79]:
if my_dump == True:
    pickle.dump(medications_lifetime, open("pickle/medications_lifetime.p", "wb"))
    pickle.dump(medications_lifetime_cost, open("pickle/medications_lifetime_cost.p", "wb"))
    pickle.dump(medications_lifetime_perc_covered, open("pickle/medications_lifetime_perc_covered.p", "wb"))
    pickle.dump(medications_lifetime_length, open("pickle/medications_lifetime_length.p", "wb"))
    pickle.dump(medications_lifetime_dispenses, open("pickle/medications_lifetime_dispenses.p", "wb"))
    pickle.dump(medications_active, open("pickle/medications_active.p", "wb"))

In [80]:
if my_pickle == True:
    medications_lifetime = pickle.load(open("pickle/medications_lifetime.p", "rb"))
    medications_lifetime_cost = pickle.load(open("pickle/medications_lifetime_cost.p", "rb"))
    medications_lifetime_perc_covered = pickle.load(open("pickle/medications_lifetime_perc_covered.p", "rb"))
    medications_lifetime_length = pickle.load(open("pickle/medications_lifetime_length.p", "rb"))
    medications_lifetime_dispenses = pickle.load(open("pickle/medications_lifetime_dispenses.p", "rb"))
    medications_active = pickle.load(open("pickle/medications_active.p", "rb"))

## Observations

Remove observations after 2020.

In [81]:

observations = observations.drop(
    observations[observations["DATE"] >= pd.to_datetime('2020-01-01')]
    .index)

observations = observations.drop(
    observations[observations["PATIENT"].isin(dead_patients)]
    .index
)




In [82]:
# get most recent observation
obs_nominal_data = (
    observations[["DATE", "PATIENT", "CODE", "VALUE", "DESCRIPTION","TYPE"]]
    .loc[observations["TYPE"] == 'text',:]
    .sort_values("DATE")
    .groupby(["PATIENT", "DESCRIPTION"])
    .tail(1)
)
obs_nominal_data["DESC"] = (
    obs_nominal_data["DESCRIPTION"].str
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
)
obs_nominal_data["VALUE"] = (
    obs_nominal_data["VALUE"].str
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
)
obs_nominal_data["PROPS"] = obs_nominal_data["DESC"].astype(str) + obs_nominal_data["VALUE"].astype(str)
obs_props = list(obs_nominal_data["PROPS"].unique())
obs_tuples = frozenset(zip(obs_nominal_data["PATIENT"], obs_nominal_data["PROPS"]))

print(len(obs_props))

#####
obs_nominal_names = list(obs_nominal_data["DESC"].unique())
obs_nominal_dict = (
    obs_nominal_data
    .pivot(index="PATIENT", columns="DESC", values="VALUE")
    .stack()
    .to_dict()
)
#######

# use get_dummies to convert to one-hot
#obs_nominal_data = pd.get_dummies(obs_nominal_data, dummy_na=True)

#obs_nominal_data.to_csv("train_nominal_dummies.csv",quoting=csv.QUOTE_NONNUMERIC)
#obs_nominal_data.head()

72


In [83]:
if my_pickle == False:
    # get most recent observation
    obs_data = (
        observations[["DATE", "PATIENT", "CODE", "VALUE", "DESCRIPTION","TYPE"]]
        .loc[observations["TYPE"] == 'numeric',:]
        .sort_values("DATE")
        .groupby(["PATIENT", "DESCRIPTION"])
        .tail(1)
    )

In [84]:
 if my_pickle == False:
    # get lifetime average of continuous observations
    obs_mean_data = (
        observations[["PATIENT", "CODE", "VALUE", "DESCRIPTION", "TYPE"]]
        .loc[observations["TYPE"] == "numeric", :]
        .groupby(["PATIENT", "DESCRIPTION"])
        .apply(lambda x: x["VALUE"].astype(float).mean()) 
        .reset_index()
        .rename(columns={0:"VALUE"})
    )



In [85]:
if my_pickle == False:
    # use eligible descriptions
    obs_data["DESC"] = (
        obs_data["DESCRIPTION"].str
        .replace(" ", "_", regex=True)
        .replace(r"\[", "_", regex=True)
        .replace(r"\.", "_", regex=True)
        .replace(r"\(","_", regex=True)
        .replace(r"-","_", regex=True)
        .replace(r"\/","_", regex=True)
        .replace(r"\]","_", regex=True)
        .replace(r"\)","_", regex=True)
        .replace(r"\+","_", regex=True)
        .replace(r"\#","_", regex=True)
    )
    obs_numeric = list(obs_data["DESC"].unique())
    obs_mean_data["DESC"] = (
        obs_mean_data["DESCRIPTION"].str
        .replace(" ", "_", regex=True)
        .replace(r"\[", "_", regex=True)
        .replace(r"\.", "_", regex=True)
        .replace(r"\(","_", regex=True)
        .replace(r"-","_", regex=True)
        .replace(r"\/","_", regex=True)
        .replace(r"\]","_", regex=True)
        .replace(r"\)","_", regex=True)
        .replace(r"\+","_", regex=True)
        .replace(r"\#","_", regex=True)
    )
    obs_mean_data["DESC"] = "mean_" + obs_mean_data["DESC"].astype(str)
    obs_numeric_mean = list(obs_mean_data["DESC"].unique())



In [86]:
if my_dump == True:
    pickle.dump(obs_data, open("pickle/obs_data.p","wb"))
    pickle.dump(obs_mean_data, open("pickle/obs_mean_data.p","wb"))
    pickle.dump(obs_numeric, open("pickle/obs_numeric.p","wb"))
    pickle.dump(obs_numeric_mean, open("pickle/obs_numeric_mean.p","wb"))

In [87]:
if my_pickle == True:
    obs_data = pickle.load(open("pickle/obs_data.p","rb"))
    obs_mean_data = pickle.load(open("pickle/obs_mean_data.p","rb"))
    obs_numeric = pickle.load(open("pickle/obs_numeric.p","rb"))
    obs_numeric_mean = pickle.load(open("pickle/obs_numeric_mean.p","rb"))
print(len(obs_numeric))
print(len(obs_numeric_mean))


139
139


In [88]:
 if my_pickle == True:
    obs_dict = (
        obs_data[["PATIENT", "DESC", "VALUE"]]
        .pivot(index="PATIENT", columns="DESC", values="VALUE")
        .stack()
        .to_dict()
    )

    obs_mean_dict = (
        obs_mean_data[["PATIENT", "DESC", "VALUE"]]
        .pivot(index="PATIENT", columns="DESC", values="VALUE")
        .stack()
        .to_dict()
    )


In [89]:
if my_dump == True:
    pickle.dump(obs_dict, open("pickle/obs_dict.p","wb"))
    pickle.dump(obs_mean_dict, open("pickle/obs_mean_dict.p","wb"))

In [90]:
obs_dict = pickle.load(open("pickle/obs_dict.p","rb"))
obs_mean_dict = pickle.load(open("pickle/obs_mean_dict.p","rb"))

## Procedures

Drop procedures from 2020.  Count lifetime procedures.  Add procedure cost.

In [91]:
procedures = procedures.drop(
    procedures[procedures["DATE"] >= pd.to_datetime('2020-01-01')]
    .index
)

procedures = procedures.drop(
    procedures[procedures["PATIENT"].isin(dead_patients)]
    .index
)

procedures_lifetime = (
    procedures[["PATIENT", "CODE"]]
    .groupby("PATIENT")
    .count()
    .loc[:,"CODE"]
)

procedures_lifetime_cost = (
    procedures[["PATIENT", "BASE_COST"]]
    .groupby("PATIENT")["BASE_COST"]
    .sum()
)

# Properties

## Allergies

In [92]:
allergy_data = (
    allergies[["STOP", "PATIENT", "DESCRIPTION"]]
    .loc[allergies["STOP"].isnull(),:]
)

# Create legible names
allergy_data["DESC"] = (
    allergy_data["DESCRIPTION"].str[:25]
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
)
allergy_names = list(allergy_data["DESC"].unique())
allergy_tuples = frozenset(zip(allergy_data["PATIENT"], allergy_data["DESC"]))
print(len(allergy_names))

15


## Devices

In [93]:
device_data = (
    devices[["STOP", "PATIENT", "DESCRIPTION"]]
    .loc[devices["STOP"].isnull(),:]
)

# Create legible names
device_data["DESC"] = (
    device_data["DESCRIPTION"].str[:25]
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
)
device_names = list(device_data["DESC"].unique())
device_tuples = frozenset(zip(device_data["PATIENT"], device_data["DESC"]))
print(len(device_names))

3


## Active Conditions

In [94]:
# Check active conditions in test file
conditions_test = pd.read_csv("test/conditions.csv",
                          parse_dates=["START","STOP"])

cond_count = (
    conditions_test[["STOP", "DESCRIPTION","CODE"]]
        .loc[conditions_test["STOP"].isnull()]
        .groupby("CODE")
        .count()
        .reset_index()
        .sort_values(by="DESCRIPTION", ascending=False)
)

# keep active conditions where there are at least 1 patients in the test set
cond_keep = cond_count.loc[cond_count["DESCRIPTION"] >= 1,"CODE"]

cond_data = (
    conditions[["STOP", "PATIENT", "CODE", "DESCRIPTION"]]
    .loc[conditions["CODE"].isin(cond_keep) & 
         conditions["STOP"].isnull(),:]
)

# Create legible names
cond_data["DESC"] = (
    cond_data["DESCRIPTION"].str[:25]
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
)
cond_names = list(cond_data["DESC"].unique())

cond_tuples = frozenset(zip(cond_data["PATIENT"], cond_data["DESC"]))

print(len(cond_names))

103


## Immunizations

In [95]:
# Create legible names
immunizations["DESC"] = (
    immunizations["DESCRIPTION"].str[:25]
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
    
)
immunization_names = list(immunizations["DESC"].unique())
immunization_tuples = frozenset(zip(immunizations["PATIENT"], immunizations["DESC"]))
print(len(immunization_names))

8


## Procedures

In [96]:
# Create legible names
procedures["DESC"] = (
    procedures["DESCRIPTION"].str[:25]
    .replace(" ", "_", regex=True)
    .replace(r"\[", "_", regex=True)
    .replace(r"\.", "_", regex=True)
    .replace(r"\(","_", regex=True)
    .replace(r"-","_", regex=True)
    .replace(r"\/","_", regex=True)
    .replace(r"\]","_", regex=True)
    .replace(r"\)","_", regex=True)
    .replace(r"\+","_", regex=True)
    .replace(r"\#","_", regex=True)
)
procedure_names = list(procedures["DESC"].unique())
procedure_tuples = frozenset(zip(procedures["PATIENT"], procedures["DESC"]))

In [97]:
print(len(procedure_names))

161


Utilities

In [98]:
def convert_name(name):
    for i in range(26):
        name = name.replace('({})'.format(chr(ord('a') + i)), '')
    name = name.replace(' ', '_')
    textform_first = {
        '^2': '_squared',
        '^3': '_cubed',
        '1/': 'inverse_of_',
        '<=': '_leq_',
        '>=': '_geq_'
    }
    textform = {
        '<': '_lt_',
        '>': '_gt_',
        '+': '_plus_',
        '-': '_minus_',
        '*': '_times_',
        '/': '_divided_by_',
        '^': '_to_the_power_',
        '(': 'open_bracket_',
        ')': '_close_bracket',
        ',': '_or_', # added by Paul
        '\'': '', # added by Paul
        '=': '_equal_' # added by Paul
               }
    for op in textform_first:
        name = name.replace(op, textform_first[op])
    for op in textform:
        name = name.replace(op, textform[op])
    return name

def convert_name_back(name):
    for i in range(26):
        name = name.replace('({})'.format(chr(ord('a') + i)), '')
    # name = name.replace('_', ' ')
    textform_first = {
        '_squared': '^2',
        '_cubed': '^3',
        'inverse_of_': '1/',
         '_leq_': '<=',
         '_geq_': '>='
    }
    textform = {
        '_lt_': '<',
        '_gt_': '>',
        '_plus_': '+',
        '_minus_': '-',
        '_times_': '*',
        '_divided_by_': '/',
        '_to_the_power_': '^',
        'open_bracket_': '(',
        '_close_bracket': ')',
        '_or_': ',', # added by Paul
        '_equal_': '=' # added by Paul
               }
    for op in textform_first:
        name = name.replace(op, textform_first[op])
    for op in textform:
        name = name.replace(op, textform[op])
    return name

def convert_conjecture_names(conjectures):
    for conj in conjectures:
        conj.__name__ = convert_name(conj.__name__)

def convert_names_back(conjectures): #note the plural name(s)
    for conj in conjectures:
        conj.__name__ = convert_name_back(conj.__name__)


# Define Patient class.

In [99]:
class Patient():
    def __init__(self, row):
        self.Id = row.Id
    ##################
    # target-related #
    ##################
    def hospitalized_status(self):
        if self.Id in inpatient_ids:
            return(True)
        return(False)
    def icu_status(self):
        if self.Id in icu_ids:
            return(True)
        return(False)
    #####################
    # target properties and invariants #
    #####################
    def covid_status(self):
        if self.Id in covid_patient_ids:
            return(True)
        return(False)
    def vent_status(self):
        if self.Id in vent_ids:
            return(True)
        return(False)
    def covid_death_status(self):
        if self.Id in deceased_ids:
            return(True)
        return(False)
    def hospital_days(self):
        if self.Id in hospital_days:
            return(float(hospital_days[self.Id]))
        return(float(0))
    def icu_days(self):
        if self.Id in icu_days:
            return(float(icu_days[self.Id]))
        return(float(0))
    ################
    # invariants   #
    ################
    def healthcare_expenses(self):
        return(float(patients_dict["HEALTHCARE_EXPENSES"][self.Id]))
    def healthcare_coverage(self):
        return(float(patients_dict["HEALTHCARE_COVERAGE"][self.Id]))
    def latitude(self):
        return(float(patients_dict["LAT"][self.Id]))
    def longitude(self):
        return(float(patients_dict["LON"][self.Id]))
    def age(self):
        return(float(patients_dict["Age"][self.Id]))
    def num_allergies(self):
        if self.Id in vets_with_allergies:
            return(float(num_allergies_dict["START"][self.Id]))
        return(float(0))
    def active_care_plans(self):
        if self.Id in active_care_plans:
            return(float(active_care_plans[self.Id]))
        return(float(0))
    def lifetime_care_plans(self):
        if self.Id in lifetime_care_plans:
            return(float(lifetime_care_plans[self.Id]))
        return(float(0))
    def active_care_plan_length(self):
        if self.Id in active_care_plan_length:
            return(float(active_care_plan_length[self.Id]))
        return(float(0))
    def lifetime_care_plan_length(self):
        if self.Id in lifetime_care_plan_length:
            return(float(lifetime_care_plan_length[self.Id]))
        return(float(0))
    def active_conditions(self):
        if self.Id in active_conditions:
            return(float(active_conditions[self.Id]))
        return(float(0))
    def lifetime_conditions(self):
        if self.Id in lifetime_conditions:
            return(float(lifetime_conditions[self.Id]))
        return(float(0))
    def active_condition_length(self):
        if self.Id in active_condition_length:
            return(float(active_condition_length[self.Id]))
        return(float(0))
    def lifetime_condition_length(self):
        if self.Id in lifetime_condition_length:
            return lifetime_condition_length[self.Id]
        return(float(0))
    def device_lifetime_length(self):
        if self.Id in device_lifetime_length:
            return(float(device_lifetime_length[self.Id]))
        return(float(0))
    def encounters_count(self):
        if self.Id in encounters_count:
            return(float(encounters_count[self.Id]))
        return(float(0))
    def encounters_lifetime_total_cost(self):
        if self.Id in encounters_lifetime_total_cost:
            return(float(encounters_lifetime_total_cost[self.Id]))
        return(float(0))
    def encounters_lifetime_base_cost(self):
        if self.Id in encounters_lifetime_base_cost:
            return(float(encounters_lifetime_base_cost[self.Id]))
        return(float(0))
    def encounters_lifetime_payer_coverage(self):
        if self.Id in encounters_lifetime_payer_coverage:
            return(float(encounters_lifetime_payer_coverage[self.Id]))
        return(float(0))
    def encounters_lifetime_perc_covered(self):
        if self.Id in encounters_lifetime_perc_covered:
            return(float(encounters_lifetime_perc_covered[self.Id]))
        return(float(0))
    def imaging_studies_lifetime(self):
        if self.Id in imaging_studies_lifetime:
            return(float(imaging_studies_lifetime[self.Id]))
        return(float(0))
    def immunizations_lifetime(self):
        if self.Id in immunizations_lifetime:
            return(float(immunizations_lifetime[self.Id]))
        return(float(0))
    def immunizations_lifetime_cost(self):
        if self.Id in immunizations_lifetime_cost:
            return(float(immunizations_lifetime_cost[self.Id]))
        return(float(0))
    def medications_lifetime(self):
        if self.Id in medications_lifetime:
            return(float(medications_lifetime[self.Id]))
        return(float(0))
    def medications_lifetime_cost(self):
        if self.Id in medications_lifetime_cost:
            return(float(medications_lifetime_cost[self.Id]))
        return(float(0))
    def medications_lifetime_perc_covered(self):
        if self.Id in medications_lifetime_perc_covered:
            return(float(medications_lifetime_perc_covered[self.Id]))
        return(float(0))
    def medications_lifetime_length(self):
        if self.Id in medications_lifetime_length:
            return(float(medications_lifetime_length[self.Id]))
        return(float(0))
    def medications_lifetime_dispenses(self):
        if self.Id in medications_lifetime_dispenses:
            return(float(medications_lifetime_dispenses[self.Id]))
        return(float(0))
    def medications_active(self):
        if self.Id in medications_active:
            return(float(medications_active[self.Id]))
        return(float(0))
    def procedures_lifetime(self):
        if self.Id in procedures_lifetime:
            return(float(procedures_lifetime[self.Id]))
        return(float(0))
    def procedures_lifetime_cost(self):
        if self.Id in procedures_lifetime_cost:
            return(float(procedures_lifetime_cost[self.Id]))
        return(float(0))
    
target_properties_names = ["covid_status", 
                           "vent_status", 
                           "covid_death_status", 
                           "hospitalized_status", 
                           "icu_status"]
target_invariants_names = ["hospital_days", "icu_days"]
    
properties_names= (
    allergy_names+
    cond_names+
    device_names+
    immunization_names+
    obs_props+
    procedure_names
)

invariants_names =  ["healthcare_expenses",
                     "healthcare_coverage",
                     "latitude",
                     "longitude",
                     "age",
                     "num_allergies",
                     "active_care_plans",
                     "lifetime_care_plans",
                     "active_care_plan_length",
                     "lifetime_care_plan_length",
                     "active_conditions",
                     "lifetime_conditions",
                     "active_condition_length",
                     "lifetime_condition_length",
                     "device_lifetime_length",
                     "encounters_count",
                     "encounters_lifetime_total_cost",
                     "encounters_lifetime_base_cost",
                     "encounters_lifetime_payer_coverage",
                     "encounters_lifetime_perc_covered ",
                     "imaging_studies_lifetime",
                     "immunizations_lifetime",
                     "immunizations_lifetime_cost",
                     "medications_lifetime",
                     "medications_lifetime_cost",
                     "medications_lifetime_perc_covered",
                     "medications_lifetime_length",
                     "medications_lifetime_dispenses",
                     "medications_active",
                     "procedures_lifetime",
                     "procedures_lifetime_cost"]

for name in obs_numeric:
    invariants_names.append(name)
for name in obs_numeric_mean:
    invariants_names.append(name)
    
# Build allergy properties
def build_allergy_prop(i):
    def prop(self):
        if (self.Id, allergy_names[i]) in allergy_tuples:
            return(True)
        return(False)
    prop.__name__ = convert_name(allergy_names[i])
    return prop

for i, name in enumerate(allergy_names):
    prop = build_allergy_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# Build device properties
def build_device_prop(i):
    def prop(self):
        if (self.Id, device_names[i]) in device_tuples:
            return(True)
        return(False)
    prop.__name__ = convert_name(device_names[i])
    return prop

for i, name in enumerate(device_names):
    prop = build_device_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# Build condition properties
def build_cond_prop(i):
    def prop(self):
        if (self.Id, cond_names[i]) in cond_tuples:
            return(True)
        return(False)
    prop.__name__ = convert_name(cond_names[i])
    return prop

for i, name in enumerate(cond_names):
    prop = build_cond_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# Build immunization properties
def build_immunization_prop(i):
    def prop(self):
        if (self.Id, immunization_names[i]) in immunization_tuples:
            return(True)
        return(False)
    prop.__name__ = convert_name(immunization_names[i])
    return prop

for i, name in enumerate(immunization_names):
    prop = build_immunization_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# Build observation properties
def build_obs_prop(i):
    def prop(self):
        if (self.Id, obs_props[i]) in obs_tuples:
            return(True)
        return(False)
    prop.__name__ = convert_name(obs_props[i])
    return prop

for i, name in enumerate(obs_props):
    prop = build_obs_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# Build procedure properties
def build_procedure_prop(i):
    def prop(self):
        if (self.Id, procedure_names[i]) in procedure_tuples:
            return(True)
        return(False)
    prop.__name__ = convert_name(procedure_names[i])
    return prop

for i, name in enumerate(procedure_names):
    prop = build_procedure_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# Build observation invariants
def build_obs_inv(i):
    def inv(self):
        try:
            return(float(obs_dict[self.Id,obs_numeric[i]]))
        except:
            return(float("NaN"))
    inv.__name__ = convert_name(obs_numeric[i])
    return inv

for i, name in enumerate(obs_numeric):
    inv = build_obs_inv(i)
    setattr(Patient, inv.__name__, inv)

def build_obs_mean_inv(i):
    def inv(self):
        try:
            return(float(obs_mean_dict[self.Id,obs_numeric_mean[i]]))
        except:
            return(float("NaN"))
    inv.__name__ = convert_name(obs_numeric_mean[i])
    return inv
    
for i, name in enumerate(obs_numeric_mean):
    inv = build_obs_mean_inv(i)
    setattr(Patient, inv.__name__, inv)
        
# Build observation nominal properties
def build_obs_nom_prop(i):
    def prop(self):
        try:
            return(str(obs_nominal_dict[self.Id, obs_nominal_names[i]]))
        except:
            return(float("NaN"))
    prop.__name__ = convert_name(obs_nominal_names[i])
    return prop

for i, name in enumerate(obs_nominal_names):
    prop = build_obs_nom_prop(i)
    setattr(Patient, prop.__name__, prop)
    
# remove special characters from property names; invariants and targets should be okay    
for i, name in enumerate(properties_names):
    properties_names[i] = convert_name(properties_names[i])

Define examples - one for each patient.

In [100]:
p_examples = patients.apply(func=Patient, 
                            axis='columns')

# Write data.

Get list of invariants.

In [101]:
target_invariants = []
target_properties = []
invariants = []
properties = []


for i in target_invariants_names:
    target_invariants.append(Patient.__dict__[i])
for i in target_properties_names:
    target_properties.append(Patient.__dict__[i])
for i in invariants_names:
    invariants.append(Patient.__dict__[i])
for i in properties_names:
    properties.append(Patient.__dict__[i])
print(len(invariants))
print(len(properties))


309
362


In [102]:
# out_data = []

# out_data_names = ["Id"]
# for j in target_properties:
#     out_data_names.append(j.__name__)
# for j in target_invariants:
#     out_data_names.append(j.__name__)
# for j in properties:
#     out_data_names.append(j.__name__)
# for j in invariants:
#     out_data_names.append(j.__name__)

# out_data.append(out_data_names)
# for i in range(len(patients)):
#     if i % 1000 == 0:
#         sys.stdout.write("%d " % int(i))    
#     this_out = [p_examples.iloc[int(i)].Id]
#     for j in target_properties:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     for j in target_invariants:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     for j in properties:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     for j in invariants:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     sys.stdout.flush()
#     out_data.append(this_out)

# with open("train.csv", "w", newline="") as trainfile:
#     writer = csv.writer(trainfile)
#     writer.writerows(out_data)
# trainfile.close()

Write data with nominal features.

In [103]:
# properties_nom_names= (
#     allergy_names+
#     cond_names+
#     device_names+
#     immunization_names+
#     obs_nominal_names+
#     procedure_names
# )

# properties_nom = []

# for i in properties_nom_names:
#     for j in Patient.__dict__:
#         if i == j:
#             properties_nom.append(Patient.__dict__[j])

# out_data = []

# out_data_names = ["Id"]
# for j in target_properties:
#     out_data_names.append(j.__name__)
# for j in target_invariants:
#     out_data_names.append(j.__name__)
# for j in properties_nom:
#     out_data_names.append(j.__name__)
# for j in invariants:
#     out_data_names.append(j.__name__)

# out_data.append(out_data_names)
# for i in range(len(patients)):
#     if i % 1000 == 0:
#         sys.stdout.write("%d " % int(i))    
#     this_out = [p_examples.iloc[int(i)].Id]
#     for j in target_properties:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     for j in target_invariants:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     for j in properties_nom:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     for j in invariants:
#         this_out.append(j(p_examples.iloc[int(i)]))
#     sys.stdout.flush()
#     out_data.append(this_out)

# with open("train_nom.csv", "w", newline="") as trainfile:
#     writer = csv.writer(trainfile)
#     writer.writerows(out_data)
# trainfile.close()

            

# Conjecturing

In [104]:
load("conjecturing.py")

## Covid Death Status among the Entire Population

In [105]:
p_examples_list = list(p_examples)

# only pick people with covid
covid_dead = [patient for patient in p_examples_list if (patient.covid_death_status())]
covid_alive= [patient for patient in p_examples_list if (not patient.covid_death_status())]

covid_invariants = invariants


set_random_seed(12345)
covid_dead_properties = []
covid_alive_properties = []
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil', 'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^'}



# not dead patients
print("Alive")
for inv in covid_invariants:
    print(inv.__name__)
    inv_of_interest = covid_invariants.index(inv)
    for i in range(3):
        # upper bounds
        conjs = conjecture(sample(covid_alive, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=True, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_alive_properties += conjs
        # lower bounds
        conjs = conjecture(sample(covid_alive, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=False, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_alive_properties += conjs
count = 0
for conj in covid_alive_properties:
    count +=1
    #print(count, convert_name_back(conj.__name__))

#  dead patients
print("Dead")
for inv in covid_invariants:
    print(inv.__name__)
    inv_of_interest = covid_invariants.index(inv)
    for i in range(3):
        # upper bounds
        conjs = conjecture(sample(covid_dead, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=True, 
                           debug=False) 
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_dead_properties += conjs
        # lower bounds
        conjs = conjecture(sample(covid_dead, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=False, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_dead_properties += conjs
count = 0
for conj in covid_dead_properties:
    count +=1
    #print(count, convert_name_back(conj.__name__))


Alive
healthcare_expenses
healthcare_coverage
latitude
longitude
age
num_allergies
active_care_plans
lifetime_care_plans
active_care_plan_length
lifetime_care_plan_length
active_conditions
lifetime_conditions
active_condition_length
lifetime_condition_length
device_lifetime_length
encounters_count
encounters_lifetime_total_cost
encounters_lifetime_base_cost
encounters_lifetime_payer_coverage
encounters_lifetime_perc_covered
imaging_studies_lifetime
immunizations_lifetime
immunizations_lifetime_cost
medications_lifetime
medications_lifetime_cost
medications_lifetime_perc_covered
medications_lifetime_length
medications_lifetime_dispenses
medications_active
procedures_lifetime
procedures_lifetime_cost
QOLS
QALY
DALY
Respiratory_rate
Heart_rate
Systolic_Blood_Pressure
Diastolic_Blood_Pressure
Body_Mass_Index
Body_Weight
Pain_severity___0_10_verbal_numeric_rating__Score____Reported
Body_Height
Triglycerides
Low_Density_Lipoprotein_Cholesterol
High_Density_Lipoprotein_Cholesterol
Creatinine


In [106]:
print(len(covid_dead_properties), len(covid_alive_properties))
print(len(covid_alive), len(covid_dead))

1421 1244
80376 5568


In [107]:

load("conjecturing.py")
set_random_seed(12345)
all_covid_properties = properties + covid_alive_properties + covid_dead_properties
all_covid_properties.append(Patient.covid_death_status)

target_prop = len(all_covid_properties)-1
for i in range(100):
    alive_conjs = propertyBasedConjecture(objects=sample(covid_alive,10)+sample(covid_dead,10), 
                                     properties = all_covid_properties,
                                     mainProperty=target_prop,
                                     sufficient=False)

    dead_conjs = propertyBasedConjecture(objects=sample(covid_alive,10)+sample(covid_dead,10), 
                                     properties = all_covid_properties,
                                     mainProperty=target_prop,
                                     sufficient=True)
count = 0
for p in alive_conjs:
    #print(count, ".", convert_name_back(p.__name__))
    count += 1
for p in dead_conjs:
    #print(count, ".", convert_name_back(p.__name__))
    count += 1


/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/misc/functional.py:1558: ComplexWarning: Casting complex values to real discards the imaginary part
  x = float(x)
/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:85: RuntimeWarning: overflow encountered in double_scalars
/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:187: RuntimeWarning: overflow encountered in double_scalars


In [108]:
load("prop_conjecturing.py")
print(len(alive_conjs))
for p in alive_conjs:
    my_conclusion = get_conclusion(p)
    num_false = 0
    num_alive = 0
    for patient in p_examples_list:
        try:# deal with missing values
            if my_conclusion(patient) == False:
                num_false += 1
                if patient.covid_death_status() == False:
                    num_alive += 1
        except:
            continue
    print(convert_name_back(p.__name__))
    print(num_alive/float(num_false))
    
print(len(dead_conjs))
for p in dead_conjs:
    my_premise = get_premise(p)
    num_true = 0
    num_dead = 0
    for patient in p_examples_list:
        try:# deal with missing values
            if my_premise(patient) == True:
                num_true += 1
                if patient.covid_death_status() == True:
                    num_dead += 1
        except:
            continue
    print(convert_name_back(p.__name__))
    print(num_dead/float(num_true))
    


5
healthcare_expenses_leq_10_to_the_power_medications_active_divided_by_num_allergies
(covid_death_status)->(healthcare_expenses<=10^medications_active/num_allergies)
0.9559652418976045
healthcare_expenses_leq_e_to_the_power_active_care_plan_length_divided_by_encounters_lifetime_perc_covered
(covid_death_status)->(healthcare_expenses<=e^active_care_plan_length/encounters_lifetime_perc_covered)
0.9854176573149469
healthcare_expenses_leq_e_to_the_power_active_condition_length_divided_by_medications_lifetime_length
(covid_death_status)->(healthcare_expenses<=e^active_condition_length/medications_lifetime_length)
0.9882517972996668
latitude_geq_flooropen_bracket_maximumopen_bracket_Hematocrit__Volume_Fraction__of_Blood_by_Automated_count_or_mean_DALY_close_bracket_close_bracket
(covid_death_status)->(latitude>=floor(maximum(Hematocrit__Volume_Fraction__of_Blood_by_Automated_count,mean_DALY)))
0.8996275935449548
encounters_count_leq_encounters_lifetime_payer_coverage_divided_by_latitude_min

## Covid Death Status among Those with Covid

In [109]:
p_examples_list = list(p_examples)

# only pick people with covid
covid_dead = [patient for patient in p_examples_list if (patient.covid_death_status() and patient.covid_status())]
covid_alive= [patient for patient in p_examples_list if (not patient.covid_death_status() and patient.covid_status())]

print(len(covid_dead), len(covid_alive))

5568 68129


In [110]:
covid_invariants = invariants


set_random_seed(12345)
covid_dead_properties = []
covid_alive_properties = []
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil', 'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^'}


# not dead patients
print("Alive")
for inv in covid_invariants:
    #print(inv.__name__)
    inv_of_interest = covid_invariants.index(inv)
    for i in range(3):
        # upper bounds
        conjs = conjecture(sample(covid_alive, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=True, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_alive_properties += conjs
        # lower bounds
        conjs = conjecture(sample(covid_alive, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=False, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_alive_properties += conjs
count = 0
for conj in covid_alive_properties:
    count +=1
    print(count, convert_name_back(conj.__name__))

#  dead patients
print("Dead")
for inv in covid_invariants:
    #print(inv.__name__)
    inv_of_interest = covid_invariants.index(inv)
    for i in range(3):
        # upper bounds
        conjs = conjecture(sample(covid_dead, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=True, 
                           debug=False) 
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_dead_properties += conjs
        # lower bounds
        conjs = conjecture(sample(covid_dead, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=False, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_dead_properties += conjs
count = 0
for conj in covid_dead_properties:
    count +=1
    print(count, convert_name_back(conj.__name__))
print("Number of dead, alive properties")
print(len(covid_dead_properties), len(covid_alive_properties))

load("conjecturing.py")

set_random_seed(12345)
all_covid_properties = properties + covid_alive_properties + covid_dead_properties
all_covid_properties.append(Patient.covid_death_status)

target_prop = len(all_covid_properties)-1
for i in range(100):
    alive_conjs = propertyBasedConjecture(objects=sample(covid_alive,10)+sample(covid_dead,10), 
                                     properties = all_covid_properties,
                                     mainProperty=target_prop,
                                     sufficient=False)

    dead_conjs = propertyBasedConjecture(objects=sample(covid_alive,10)+sample(covid_dead,10), 
                                     properties = all_covid_properties,
                                     mainProperty=target_prop,
                                     sufficient=True)
count = 0
for p in alive_conjs:
    #print(count, ".", convert_name_back(p.__name__))
    count += 1
for p in dead_conjs:
    #print(count, ".", convert_name_back(p.__name__))
    count += 1


load("prop_conjecturing.py")
print("Property Conjectures")
print(len(alive_conjs))
for p in alive_conjs:
    print(convert_name_back(p.__name__))
    my_conclusion = get_conclusion(p)
    num_false = 0
    num_alive = 0
    for patient in p_examples_list:
        try:  # deal with missing values
            if my_conclusion(patient) == False:
                num_false += 1
                if patient.covid_death_status() == False:
                    num_alive += 1
        except:
            continue
    print(num_alive/float(num_false))
print(len(dead_conjs))
for p in dead_conjs:
    print(convert_name_back(p.__name__))
    my_premise = get_premise(p)
    num_true = 0
    num_dead = 0
    for patient in p_examples_list:
        try: # deal with missing values
            if my_premise(patient) == True:
                num_true += 1
                if patient.covid_death_status() == True:
                    num_dead += 1
        except:
            continue
    print(num_dead/float(num_true))

Alive
1 healthcare_expenses<=healthcare_coverage^active_conditions*medications_lifetime_dispenses
2 healthcare_expenses<=(medications_lifetime_cost-1)*QALY
3 healthcare_expenses<=(e^QOLS)^QALY
4 healthcare_expenses<=(QOLS+1)^latitude
5 healthcare_expenses<=10^floor(sqrt(age))
6 healthcare_expenses<=1/2*10^sqrt(latitude)
7 healthcare_expenses<=1/16*longitude^4
8 healthcare_expenses>=healthcare_coverage*log(lifetime_condition_length)/log(10)
9 healthcare_expenses>=medications_lifetime_length^2/medications_lifetime^2
10 healthcare_expenses>=(procedures_lifetime_cost+1)/QOLS
11 healthcare_expenses>=encounters_lifetime_payer_coverage^2/active_conditions^2
12 healthcare_expenses>=age^2*medications_lifetime
13 healthcare_expenses>=sqrt(QOLS)*medications_lifetime_cost
14 healthcare_expenses>=10^(active_conditions/medications_lifetime)
15 healthcare_expenses>=(QALY-1)*encounters_lifetime_total_cost
16 healthcare_expenses>=DALY^2*procedures_lifetime_cost
17 healthcare_expenses<=e^QALY/mean_QALY


/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:85: RuntimeWarning: overflow encountered in double_scalars
/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/misc/functional.py:1558: ComplexWarning: Casting complex values to real discards the imaginary part
  x = float(x)
/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:187: RuntimeWarning: overflow encountered in double_scalars


Property Conjectures
6
(covid_death_status)->(healthcare_coverage>=healthcare_expenses*medications_lifetime_perc_covered/active_condition_length)
healthcare_coverage_geq_healthcare_expenses_times_medications_lifetime_perc_covered_divided_by_active_condition_length
0.9719121683440073
(covid_death_status)->(healthcare_expenses>=medications_lifetime_length^2/medications_lifetime^2)
healthcare_expenses_geq_medications_lifetime_length_squared_divided_by_medications_lifetime_squared
0.9774005866230706
(covid_death_status)->(healthcare_expenses<=1/2*encounters_lifetime_total_cost*healthcare_coverage)
healthcare_expenses_leq_inverse_of_2_times_encounters_lifetime_total_cost_times_healthcare_coverage
0.986516000379831
(covid_death_status)->(healthcare_coverage<=10^active_conditions/active_care_plans)
healthcare_coverage_leq_10_to_the_power_active_conditions_divided_by_active_care_plans
0.9712770216172938
(covid_death_status)->(active_condition_length>=1/2*encounters_lifetime_perc_covered*lifeti

## ICU Status among Those with Covid

In [111]:
p_examples_list = list(p_examples)

# only pick people with covid
covid_not_icu = [patient for patient in p_examples_list if (patient.icu_status() and patient.covid_status())]
covid_icu= [patient for patient in p_examples_list if (not patient.icu_status() and patient.covid_status())]

print(len(covid_not_icu), len(covid_icu))

4981 68716


In [112]:
covid_invariants = invariants


set_random_seed(12345)
covid_icu_properties = []
covid_not_icu_properties = []
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil', 'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^'}


# not dead patients
print("ICU")
for inv in covid_invariants:
    #print(inv.__name__)
    inv_of_interest = covid_invariants.index(inv)
    for i in range(3):
        # upper bounds
        conjs = conjecture(sample(covid_icu, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=True, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_icu_properties += conjs
        # lower bounds
        conjs = conjecture(sample(covid_icu, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=False, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_icu_properties += conjs
count = 0
for conj in covid_icu_properties:
    count +=1
    print(count, convert_name_back(conj.__name__))

#  dead patients
print("Not ICU")
for inv in covid_invariants:
    #print(inv.__name__)
    inv_of_interest = covid_invariants.index(inv)
    for i in range(3):
        # upper bounds
        conjs = conjecture(sample(covid_not_icu, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=True, 
                           debug=False) 
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_not_icu_properties += conjs
        # lower bounds
        conjs = conjecture(sample(covid_not_icu, 10), 
                           covid_invariants, 
                           inv_of_interest, 
                           operators=use_operators, 
                           upperBound=False, 
                           debug=False)
        convert_conjecture_names(conjs)
        #for c in conjs:
        #    print(c)
        covid_not_icu_properties += conjs
count = 0
for conj in covid_not_icu_properties:
    count +=1
    print(count, convert_name_back(conj.__name__))
print("Number of not ICU, ICU properties")
print(len(covid_not_icu_properties), len(covid_icu_properties))

load("conjecturing.py")

set_random_seed(12345)
all_covid_properties = properties + covid_icu_properties + covid_not_icu_properties
all_covid_properties.append(Patient.icu_status)

target_prop = len(all_covid_properties)-1
for i in range(100):
    not_icu_conjs = propertyBasedConjecture(objects=sample(covid_icu,10)+sample(covid_not_icu,10), 
                                     properties = all_covid_properties,
                                     mainProperty=target_prop,
                                     sufficient=False)

    icu_conjs = propertyBasedConjecture(objects=sample(covid_icu,10)+sample(covid_not_icu,10), 
                                     properties = all_covid_properties,
                                     mainProperty=target_prop,
                                     sufficient=True)
count = 0
for p in icu_conjs:
    #print(count, ".", convert_name_back(p.__name__))
    count += 1
for p in not_icu_conjs:
    #print(count, ".", convert_name_back(p.__name__))
    count += 1


load("prop_conjecturing.py")
print("Property Conjectures")
print(len(not_icu_conjs))
for p in not_icu_conjs:
    my_conclusion = get_conclusion(p)
    num_false = 0
    num_not_icu = 0
    for patient in p_examples_list:
        try: 
            if my_conclusion(patient) == False:
                num_false += 1
                if patient.icu_status() == False:
                    num_not_icu += 1
        except:
            continue
    print(convert_name_back(p.__name__))
    print(num_not_icu/float(num_false))
print(len(icu_conjs))
for p in icu_conjs:
    my_premise = get_premise(p)
    num_true = 0
    num_icu = 0
    for patient in p_examples_list:
        try: # deal with missing values
            if my_premise(patient) == True:
                num_true += 1
                if patient.icu_status() == True:
                    num_icu += 1
        except:
            continue
    print(convert_name_back(p.__name__))
    print(num_icu/float(num_true))

ICU
1 healthcare_expenses<=1/2*(healthcare_coverage-1)^2
2 healthcare_expenses<=1/16*longitude^4
3 healthcare_expenses<=active_condition_length*healthcare_coverage/num_allergies
4 healthcare_expenses<=healthcare_coverage*lifetime_care_plan_length/medications_lifetime_perc_covered
5 healthcare_expenses<=2*(encounters_lifetime_total_cost-1)^2
6 healthcare_expenses<=10^active_care_plan_length/DALY
7 healthcare_expenses<=4*e^lifetime_condition_length
8 healthcare_expenses<=10^(latitude/medications_active)
9 healthcare_expenses<=10^abs(log(procedures_lifetime_cost))
10 healthcare_expenses>=10^(latitude/lifetime_condition_length)
11 healthcare_expenses>=2*10^(1/encounters_lifetime_perc_covered)
12 healthcare_expenses>=log(encounters_count)*medications_lifetime_cost/log(10)
13 healthcare_expenses>=(2*encounters_lifetime_perc_covered)^latitude
14 healthcare_expenses>=(medications_lifetime_cost+1)/encounters_lifetime_perc_covered
15 healthcare_expenses>=(age+1)*procedures_lifetime_cost
16 healt

/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:187: RuntimeWarning: overflow encountered in double_scalars
/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:40: RuntimeWarning: overflow encountered in exp
/Users/jpbrooks/opt/anaconda3/envs/sage/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:171: RuntimeWarning: overflow encountered in double_scalars


Property Conjectures
7
healthcare_coverage_geq_e_to_the_power_open_bracket_2_times_10_to_the_power_medications_lifetime_perc_covered_close_bracket
(icu_status)->(healthcare_coverage>=e^(2*10^medications_lifetime_perc_covered))
0.9337755919020931
healthcare_expenses_leq_e_to_the_power_open_bracket__minus_DALY_plus_QALY_close_bracket
(icu_status)->(healthcare_expenses<=e^(-DALY+QALY))
0.9349005424954792
healthcare_expenses_geq_procedures_lifetime_to_the_power_e_to_the_power_immunizations_lifetime
(icu_status)->(healthcare_expenses>=procedures_lifetime^e^immunizations_lifetime)
0.9244444444444444
healthcare_coverage_leq_open_bracket_encounters_lifetime_total_cost_minus_1_close_bracket_times_age
(icu_status)->(healthcare_coverage<=(encounters_lifetime_total_cost-1)*age)
0.9833876221498371
healthcare_coverage_geq_encounters_lifetime_total_cost_times_sqrtopen_bracket_immunizations_lifetime_close_bracket
(icu_status)->(healthcare_coverage>=encounters_lifetime_total_cost*sqrt(immunizations_lif